In [2]:
! pip install kora

     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 


In [5]:
from kora import console
console.start()


Console URL: https://teleconsole.com/s/82491deebd4214671c95cc42e5ec562eff02fd20


In [5]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])

output.register_callback('shell', shell)

In [1]:
# 필요한 패키지를 설치합니다
%%capture
!pip install pororo -q
!pip install imapclient -q
! pip install pretty_html_table 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import sys
import time
import random

sys.path.append('/content/drive/MyDrive/Colab Notebooks/news_feeder')

In [4]:
from pororo import Pororo
summary = Pororo(task="summary", lang="kr")

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
 
 
# os.chdir('자신의 디렉토리 주소')
os.chdir('/content/drive/MyDrive/Colab Notebooks/news_feeder')
 
 
def crawler(company_code, num_article):
    
    done_page_num=0

    # page = 1 
    num_per_page=20 # naver serves 20 articles per page
    num_page,remainder=divmod(num_article,20)
    num_page+=1

    article_result=[]
    
    for page in range(done_page_num+1, done_page_num+num_page+1):
        try:
            url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) 
            source_code = requests.get(url).text
            html = BeautifulSoup(source_code, "lxml")
    
            
            # 뉴스 링크
            links = html.select('.title') 

            link_result =[]
            if page == num_page:
                links=links[:remainder]

            for link in links: 
                add = 'https://finance.naver.com' + link.find('a')['href']
            
                link_result.append(add)
            print(f"{len(link_result)}개의 뉴스 크롤링..")
        except Exception:
            pass 
        # print(len(link_result))

        
        # 뉴스 날짜 
        # dates = html.select('.date') 
        # date_result = [date.get_text() for date in dates] 
        
        # cnt=0
        for article_url in link_result: 
            try:
                # cnt+=1
                # if cnt%100==0:
                #     print(f"{cnt}번째 기사 처리중")
            #    article_url = link_result[0] 
                article_source_code = requests.get(article_url).text
                article_html = BeautifulSoup(article_source_code, "lxml")
                article_time = article_html.select('.tah')[0].get_text()

                # 뉴스 내용
                article_contents = article_html.select('.scr01')
                article_contents=article_contents[0].get_text()
                article_contents = re.sub('\n','',article_contents)
                article_result.append((article_contents,article_time))

                time.sleep(random.uniform(0.1,0.7))
            except Exception:
                pass

        # print("다운 받고 있습니다------")

    return article_result

In [6]:
# 종목 리스트 파일 열기  
# 회사명을 종목코드로 변환 
        
def convert_to_code(company, max_num=5):
    
    print(f"{company} 종목 뉴스를 가져옵니다.")

    data = pd.read_csv('company_list.txt', dtype=str, sep='\t')   # 종목코드 추출 
    company_name = data['회사명']
    keys = [i for i in company_name]    #데이터프레임에서 리스트로 바꾸기 
 
    company_code = data['종목코드']
    values = [j for j in company_code]
 
    dict_result = dict(zip(keys, values))  # 딕셔너리 형태로 회사이름과 종목코드 묶기 
    
    pattern = '[a-zA-Z가-힣]+' 
    

In [ ]:

    if bool(re.match(pattern, company)) == True:         # Input에 이름으로 넣었을 때  
        company_code = dict_result.get(str(company))
        if company_code:
            result=crawler(company_code, max_num)
            result=list(zip([company for i in range(len(result))],result))
            return result

        else:
            print(f"{company} 종목이 존재하지 않습니다.")   
            return []

    else:                                                # Input에 종목코드로 넣었을 때       
        company_code = str(company)      
        result=crawler(company_code, max_num)
        if result:
            result=list(zip([company for i in range(len(result))],result))
            return result
        else:
            print(f"{company} 종목이 존재하지 않습니다.")  
            return []

In [7]:
company_list=["삼성전자","SK하이닉스","현대자동차"]

In [8]:
crawl_result=[]
for company in company_list:
    crawl_result+=convert_to_code(company,5)

삼성전자 종목 뉴스를 가져옵니다.
5개의 뉴스 크롤링..
SK하이닉스 종목 뉴스를 가져옵니다.
5개의 뉴스 크롤링..
현대자동차 종목 뉴스를 가져옵니다.
5개의 뉴스 크롤링..


In [15]:
text_list=crawl_result.copy()
crawl_text=[ele[1] for ele in text_list]
summary_result=[summary(text) for text in text_list]


IndexError: ignored

In [19]:
text_list

[('삼성전자',
  ('강병주 삼성카드 전무(가운데), 조장호 삼성전자 상무(왼쪽), 이태형 마스터카드 부사장(오른쪽)이 기념 사진을 촬영하고 있다.(사진=삼성카드)삼성카드는 3일 삼성전자, 마스터카드와 \'지문인증 카드\' 업무제휴 협약을 체결했다고 4일 밝혔다.각 사는 협업을 통해 지문인증 카드 개발을 진행하고 올 하반기에 상용화를 추진할 예정이다.삼성카드는 지문인증 카드의 국내 시장 도입을 담당하고 삼성전자는 지문인증 IC칩 개발 및 공급, 마스터카드는 지문인증 카드 해외 도입을 각각 맡아서 진행한다.지문인증 카드는 사용자의 지문정보를 저장하고 인증할 수 있는 IC칩이 내장된 카드다. 지문 센서에 손가락을 올린 상태에서 카드를 단말기에 삽입하거나 터치하게 되면 결제가 진행된다.해외에서 결제 시 비밀번호나 핀(PIN)번호 입력 등을 하지 않아도 되기 때문에 비밀번호 노출 등의 우려가 없고 지문 인증을 통해 본인만 결제가 가능하므로 실물 카드 도난 및 분실에 따른 결제 피해도 줄어든다.지문인증 카드는 IC카드 단말기가 설치된 국내 및 해외 가맹점에서 모두 사용할 수 있어 상용화에 용이하다. 또한 지문인증 카드는 결제단말기에서 전원을 공급 받으므로 카드에 별도의 배터리 장착이 필요 없다는 것도 장점이다.삼성카드는 지문인증 카드를 올 하반기에 해외 카드결제 이용이 많은 법인카드에 우선 적용한 후 시장 상황에 맞춰 대상을 점차 확대해나갈 계획이다.삼성카드 관계자는 "각 사의 역량을 활용한 협업을 통해 지문인증 카드를 비롯해 고객의 편의성을 높일 수 있는 서비스를 지속적으로 제공할 계획"이라고 말했다.차은지 한경닷컴 기자 chachacha@hankyung.com▶ 경제지 네이버 구독 첫 400만, 한국경제 받아보세요▶ 한경 고품격 뉴스레터, 원클릭으로 구독하세요▶ 한국경제신문과 WSJ, 모바일한경으로 보세요 ⓒ 한국경제 & hankyung.com, 무단전재 및 재배포 금지한국경제 관련뉴스해당 언론사에서 선정하며 언론사 페이지(아웃링크)로 이동해 볼 수 있습니다."

In [17]:
[ele[0] for ele in text_list]

['삼성전자',
 '삼성전자',
 '삼성전자',
 '삼성전자',
 '삼성전자',
 'SK하이닉스',
 'SK하이닉스',
 'SK하이닉스',
 'SK하이닉스',
 'SK하이닉스',
 '현대자동차',
 '현대자동차',
 '현대자동차',
 '현대자동차',
 '현대자동차']

In [18]:
summary_result

[['대구 수성구에 위치한 한 농 농 농 농 축 축 농 농 축 농 농 농 수 농 축 축 축 축 농 축 농 축 축 농업 농업 농업 농업 농 농 수 축 축 농 수 농 농 수 수 농 수 농 수 수 적 적 적 농 농 수 적 적 농 수 수 수 수 농 농 축 수 수 농 축 농 수 수 농업 농업 농업 수 수 수 적 수 농 수 축 수 수 수 다양한 다양한 농 수 수 축 축 수 수 축 수 농 수 축산 축산 수 수 수 환경 수 수 수 각 농 수 수 다양한 수 수 수 관련 수 수 수 지역 수 수 수 축산 농 수 수 환경 환경 환경 환경 수 수 다양한 방 수 수 수 전시 수 수 수 각각 수 수 수 구분 수 수 수 안내 안내 수 수 수    수 수 수 사용 사용 사용 사용 수 수 수 지 지 지 지 수 수 사용 수 사용 사용 수 방 방 방 방 수 수 지 사용 사용 사용 각각 각각 수 수 사용 삼성전자 사용 사용 사용 삼성전자 지 지 지 사용 사용 삼성전자     지 지 지 은 은 은 사용 사용 사용 지 지 지    사용 사용 사용    은 은 은 은 지 지 지 진 진 진 진 지 지 지 S S S S 지 지 사용 지 지  지 지 사용   지   지 사용 사용  지 지  사용   사용 지 지 사용 진 진 진    열 지 지 지 전 지 지 지 도    경기 지 지 지 박 박 박    삼성전자   지 S 지 지 지 삼    S S S    전 지 지  은    삼   삼성전자 삼성전자   삼성전자 사용   진 진 지   삼성전자 지 지 S   삼성전자 박 박 박 박 지 지 지 삼성전자 삼성전자  지 지 은   지 삼성전자  삼성전자  삼성전자 삼성전자 삼성전자 지 지 삼성전자 지 지 사용 은 은 은  지 지 삼성전자   사용 은 은 사용 은   전    더불어 지 지 지 삼성   지엽 지 지 지 적 지 지 지엽  삼성전자 삼성전자 사용 삼성전자 삼성전자 삼성전자 삼성전자  삼성전자 지 삼성전자 삼성전자 지 삼성전자 지엽  지엽   삼성전자 슈퍼 슈퍼 삼성전자 삼성전자 지엽 은 은 사용   삼

In [ ]:

result=list(zip([ele[0] for ele in text_list],summary_result, [ele[2] for ele in text_list]))

In [13]:
def process_summary(text_list):
    crawl_text=[ele[1] for ele in text_list]
    summary_result=[summary(text) for text in text_list]
    return list(zip([ele[0] for ele in text_list],summary_result, [ele[2] for ele in text_list]))

In [14]:
from pretty_html_table import build_table

news_summary=process_summary(crawl_result)

news_table=pd.DataFrame(news_summary)
news_table.columns=["company","summary","date"]
table_to_send=build_table(news_table,"blue_light")

IndexError: ignored

In [23]:
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


def send_mail(subject,from_email, password ,to_email, basic_text, body):

    message = MIMEMultipart()
    message['Subject'] = subject
    message['From'] = from_email
    message['To'] = to_email

    message.attach(MIMEText(basic_text, "plain"))
    message.attach(MIMEText(body, "html"))
    msg_body = message.as_string()

    server = SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(message['From'], password)
    server.sendmail(message['From'], message['To'], msg_body)
    server.quit()


In [24]:
import getpass
from_email_id = 'mattias.colab@gmail.com'
from_email_pw = getpass.getpass(from_email_id + "'s password: ")

subject=f"{company_list[0]}외 종목의 요약 뉴스 전달해 드립니다."
from_email="mattias.colab@gmail.com"
to_email="nilsine11202@gmail.com"
basic_text="어때요 좀 오를 것 같아요?"

mattias.colab@gmail.com's password: ··········


In [25]:
send_mail(subject=subject, 
          from_email=from_email_id,
          password=from_email_pw,
          to_email=to_email,
          basic_text=basic_text,
          body=table_to_send)

In [ ]:
# 이제 1분단위 (60초)로 무한반복 
from time import sleep
import datetime

while(True):
  print("Checking again at", datetime.datetime.now())
  check_imap()
  sleep(60)

In [ ]:
# from send_email import SendEmail
# emailAddr = "mattias.colab@gmail.com"   #MK: sender email address
# emailPass = "gdgtowpjureldhcc"         #MK: password for sender email

In [62]:
#"#순이 (요약기)의 이메일. 여기서는 gmail을 사용한다. second gmail을 하나 생성하는 것이 안전하고 좋다.


mattias.colab's password: ··········


In [105]:
password

'gdgtowpjureldhcc'

In [96]:

# SSL은 살살 가도록
import ssl
# context manager ensures the session is cleaned up
ssl_context = ssl.create_default_context()

# don't check if certificate hostname doesn't match target hostname
ssl_context.check_hostname = False

# don't check if the certificate is trusted by a certificate authority
ssl_context.verify_mode = ssl.CERT_NONE


# 메일 보내기 함수, gmail의 SMTP를 사용한다.
import smtplib

def sendmail(to, title, body, inreply="new"):

    message = "Subject: " + title + \
        "\nIn-Reply-To: " + inreply + \
        "\nFrom: Auto Summary <autosumkr@gmail.com>" + \
        "\n\n" + body # + \
        # "\n---\n" + body

    with smtplib.SMTP_SSL("smtp.gmail.com", context=ssl_context) as server:
        server.login(username, password)
        server.sendmail(username, to, message.encode("utf8"))


In [ ]:
# IMAP로 메일을 가져와 요약한다
from imapclient import IMAPClient
import email

def check_imap():
    with IMAPClient(host="imap.gmail.com", ssl_context=ssl_context) as client:
        client.login(username, password)
        client.select_folder('INBOX')

        # search criteria are passed in a straightforward way
        # (nesting is supported)
        messages = client.search(['NOT', 'DELETED'])

        # fetch selectors are passed as a simple list of strings.
        response = client.fetch(messages, ['FLAGS', 'RFC822'])

        # `response` is keyed by message id and contains parsed,
        # converted response items.
        for message_id, data in response.items():
            
            email_message = email.message_from_bytes(data[b"RFC822"])
            
            efrom = email_message.get("From")
            eto = email_message.get("To")
            esubject = email_message.get("Subject")

            inreply = email_message.get("Message-ID")

            if eto is None or efrom is None:
              continue

            if eto == username + "@gmail.com":
                mailto = efrom
            else:
                mailto = eto
            
            ebody = ""
            for part in email_message.walk():
                if part.get_content_type() == 'text/plain':
                    charset = part.get_content_charset()
                    ebody = str(part.get_payload(decode=True), 'utf-8', 'ignore')
            
            #print("Sending", mailto, message_id, inreply, esubject)  
            try:          
              sendmail(mailto, esubject, ebody, inreply)

            except Exception as e:
              print(e)
              print("Skipping!", message_id)


            client.delete_messages(message_id)
            print("Sent!", message_id)
        client.expunge()
        client.logout()

In [ ]:
# 변수들 합쳐서 해당 디렉토리에 csv파일로 저장하기 

result= {"날짜" : date_result, "언론사" : source_result, "기사제목" : title_result, "링크" : link_result} 
print(result)
df_result = pd.DataFrame(result)

![image](https://i.stack.imgur.com/lkvoo.jpg)

In [ ]:
# https://stackoverflow.com/questions/57113226
while True:pass